In [80]:
import requests, pprint
from bs4 import BeautifulSoup
from datetime import datetime
url = "https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=136990&type=after&page=1"


def get_html(_url):
    res = requests.get(_url)
    html = BeautifulSoup(res.content, 'html.parser')
    """
    Function select is a copy selector reading from tag. 
        div { id: section_review, class: detail_cont detail_customer_review }
            div { id: _review_filter_container }
                ul { id: _review_list, class: lst_review }
                    li { class: thumb_nail } or li { class: thumb_nail_open } javascript function execution !
    """
    li_size = len(html.find_all("li", attrs={'class': 'thumb_nail'}))
    crawling_arr_set = []
    while li_size:
        average_star = html.select("#_review_list > li:nth-child(%d) > div > div.avg_area > a > span.curr_avg > strong" % li_size)[0].text
        store = html.select("#_review_list > li:nth-child(%d) > div > div.avg_area > span > span:nth-child(1)" % li_size)[0].text
        user_name = html.select("#_review_list > li:nth-child(%d) > div > div.avg_area > span > span:nth-child(2)" % li_size)[0].text
        write_date = html.select("#_review_list > li:nth-child(%d) > div > div.avg_area > span > span:nth-child(3)" % li_size)[0].text
        title_text = html.select("#_review_list > li:nth-child(%d) > div > p > strong" % li_size)[0].text
        body_text = html.select("#_review_list > li:nth-child(%d) > div > div.atc" % li_size)[0].text
        # print(li_size)
        # print(average_star, store, user_name, write_date, title_text, body_text)
        crawling_arr_set.append({
            "aver_star": average_star,
            "user": user_name,
            "store": store,
            "date": write_date,
            "title": title_text,
            "body": body_text
        })
        li_size -= 1
    return crawling_arr_set
    

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(get_html("https://search.shopping.naver.com/detail/detail.nhn?nvMid=21076849694&adId=nad-a001-02-000000075819074&channel=nshop.npla&query=%EA%B3%B5%EA%B8%B0%EC%B2%AD%EC%A0%95%EA%B8%B0&NaPm=ct%3Dkad8qf08%7Cci%3D0yW0002VbGbs1XUxwvkS%7Ctr%3Dpla%7Chk%3D6066b0a1499f4f77890d54202d9808691c54a75d"))


[   {   'aver_star': '5',
        'body': '익히 소문도 듣고, 직접 사용하는 곳에서 성능도 확인 및 감지하고 집에서도 사용하려고 주문했습니다~ 역시나 '
                '바로 시운전 해보니 구웃~ 더군다나 평수가 적절하다 보니 사무실에서 사용하는 것보다 더 효율이 좋을 듯 '
                '하네요~^^가성비 굿, 화이트 원기둥형의 심플하면서 세련된 디자인 굿, 하단부 튼튼한 바퀴 부착해서 이동성 '
                '굿, 손쉬운 작동 메뉴 및 버튼 굿, 상태별 변화하는 컬러로 손쉬운 실내상태 확인 굿, 손쉬운 필터 청소 및 '
                '세척필터 사용 굿, 저소음 저전력(65W) 굿, 성능은 꾸준히 사용해 봐야 알겠지만 지금까진 구웃입니다~^^ '
                '(처음 개봉 후 사용 시 꼭, 필터 확인하시고 포장비닐 제거 후에 사용하세요~)',
        'date': '20.01.17.',
        'store': '11번가',
        'title': '최고예요',
        'user': '95******'},
    {   'aver_star': '5',
        'body': '빠른 배송 정말 만족합니다!!! (2일 만에 도착했어요^^)공기청정기 구매하려고 여러 제품을 많이 찾아봤는데 '
                '(오프라인,온라인) 가성비가 좋다는 후기가 많아 쿠쿠 공기청정기를 선택하게 되었습니다.무엇보다 맘에 들었던 건 '
                '360도 공기청정이라는 점과 대용량이었어요.게다가 배송 받아보니 디자인도 넘 깔끔하게 예쁘고, 바퀴가 있어 '
                '이동이 편리한게 장점 인것 같아요. 소음도 거의 없는 편이고, 선택을 넘 잘했다고 만족하고 있어요.가성비 좋은 '
                '제품을 찾는다면 쿠쿠 공기청정기 추천합니다.많이 파세요~~~',
       